In [1]:
import boto3, os, sagemaker
from sagemaker.pytorch.estimator import PyTorch
from sagemaker import get_execution_role
import time
from time import gmtime, strftime


#role =  'arn:aws:iam::474108708967:role/PowerUserAccess'
role = 'arn:aws:iam::474108708967:role/AWSSageMakerFull'
bucket = 'cockpit-bucket2'
dataset_prefix='dataset'

sess = sagemaker.Session()
region = boto3.Session().region_name
print(role)
print(region)

s3_train_data = 's3://{}/{}/train/dataset-input-train-SM.csv'.format(bucket, dataset_prefix) #dataset-input-train.csv
s3_validation_data = 's3://{}/{}/val/dataset-input-val-SM.csv'.format(bucket, dataset_prefix) #dataset-input-val.csv

arn:aws:iam::474108708967:role/AWSSageMakerFull
us-east-2


In [18]:

hyperparameters= {
    'epochs': 10,
    'batch-size': 50,
    'learning-rate': 1E-3
}
job_name_prefix = 'acopio-output'
job_name = job_name_prefix # + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
s3_output_location = 's3://{}/{}/output'.format(bucket, job_name)

pytorch_estimator = PyTorch('./acopio-train.py',
                            py_version='py3',
                            framework_version='1.5.0',
                            base_job_name=job_name,
                            role=role,
                            train_instance_type= 'ml.m5.xlarge', #'local',
                            train_instance_count=1,
                            use_spot_instances= True,
                            max_run=250,
                            max_wait=500,
                            hyperparameters = hyperparameters,
                            metric_definitions=[
                               {'Name': 'train:error', 'Regex': 'Train_error=(.*?);'},
                               {'Name': 'validation:error', 'Regex': 'Valid_error=(.*?);'}
                            ],
                            #tags: [{'project': 'acopio'}],
                            output_path=s3_output_location)

In [19]:
train_data = sagemaker.session.s3_input(s3_train_data, content_type='text/csv')
validation_data = sagemaker.session.s3_input(s3_validation_data, content_type='text/csv')

data_channels = {'train': train_data, 'validation': validation_data}

pytorch_estimator.fit(data_channels)

2020-10-30 11:34:12 Starting - Starting the training job...
2020-10-30 11:34:14 Starting - Launching requested ML instances......
2020-10-30 11:35:21 Starting - Preparing the instances for training...
2020-10-30 11:36:01 Downloading - Downloading input data...
2020-10-30 11:36:50 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-10-30 11:36:50,840 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-10-30 11:36:50,843 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-10-30 11:36:50,853 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-10-30 11:36:51,093 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-10-30 11:36:51,357 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 


In [20]:
from time import strftime, gmtime
timestamp = strftime('%d-%H-%M-%S', gmtime())

endpoint_name = 'acopio-'+timestamp
print(endpoint_name)

acopio-30-11-39-43


In [21]:
pytorch_predictor = pytorch_estimator.deploy(endpoint_name=endpoint_name, 
                        initial_instance_count=1, 
                        instance_type='ml.t2.medium')

------------------!

In [22]:
"""with open('./test_endpoint.csv') as file_input:
            csv_file_input = DictReader(file_input, delimiter=',')
            for row in csv_file_input:
                input = []
"""
import pandas as pd
test_data = pd.read_csv('test_endpoint.csv',delimiter=',')
test_datax = test_data.to_csv(index=False)

print(test_datax)

VBELN,POSNR,PSTYV,ERNAM,ERZET,MATNR,MATWA,MATKL,WERKS,LGORT,CHARG,LICHN,KDMAT,PRODH,LFIMG,MEINS,VRKME,UMVKZ,UMVKN,NTGEW,BRGEW,GEWEI,VOLUM,VOLEH,KZTLF,UEBTK,UEBTO,UNTTO,CHSPL,FAKSP,MBDAT,LGMNG,ARKTX,LGPBE,VBELV,POSNV,VBTYV,VGSYS,VGBEL,VGPOS,UPFLU,UEPOS,FKREL,LADGR,TRAGR,KOMKZ,LGNUM,LISPL,LGTYP,LGPLA,BWTEX,BWART,BWLVS,KZDLG,BDART,PLART,MTART,XCHPF,XCHAR,VGREF,POSAR,BWTAR,SUMBD,MTVFP,EANNR,GSBER,VKBUR,VKGRP,VTWEG,SPART,GRKOR,FMENG,ANTLF,VBEAF,VBEAV,STAFO,WAVWR,KZWI1,KZWI2,KZWI3,KZWI4,KZWI5,KZWI6,SOBKZ,AEDAT,EAN11,BZIRK,VKORG,ERDAT,LFART,AUTLF,KZAZU,WADAT,LDDAT,TDDAT,LFDAT,KODAT,ABLAD,INCO1,INCO2,EXPKZ,ROUTE,FAKSK,LIFSK,VBTYP,KNFAK,TPQUA,TPGRP,LPRIO,VSBED,KUNNR,KUNAG,KDGRP,STZKL,STZZU,BTGEW,ANZPK,BEROT,LFUHR,GRULG,LSTEL,FKARV,FKDAT,PERFK,ROUTA,KALSM,KNUMV,WAERK,VBEAK,ZUKRL,VERUR,COMMN,STWAE,STCUR,EXNUM,AENAM,VKOIV,VTWIV,SPAIV,FKAIV,PIOIV,FKDIV,KUNIV,KKBER,KNKLI,GRUPP,SBGRP,CTLPC,CMWAE,AMTBL,BOLNR,LIFNR,TRATY,TRAID,CMFRE,CMNGV,XABLN,BLDAT,WADAT_IST,TRSPG,TPSID,LIFEX,TERNR,KALSM_CH,KLIEF,KAL

In [23]:
import torch
import sagemaker

from sagemaker.pytorch.model import PyTorchPredictor

#endpoint_name = 'acopio-28-17-24-54'
#pytorch_predictor = PyTorchPredictor(endpoint_name, sagemaker_session=None)

#pytorch_predictor.content_type = 'application/json'
pytorch_predictor.serializer = sagemaker.serializers.CSVSerializer()
pytorch_predictor.deserializer = sagemaker.deserializers.CSVDeserializer()

response = pytorch_predictor.predict(test_datax)
print(response[0])

['T802']


In [52]:
pytorch_predictor.delete_endpoint()

In [55]:
import torch
print(torch.version)

ModuleNotFoundError: No module named 'torch'

In [43]:
output = torch.Tensor([   122.6831,  -78.2021,   89.6559,202.9287,   76.5200,  178.9879])

_, predicted = torch.max(output.data, 0)
num = predicted.item()
tens = torch.Tensor([num])
tens = tens.long()

print(tens)

tensor([3])


In [17]:
tens = 'T802'
print([tens])

['T802']
